# Sprint 1: Formalización del problema

## Espacio de codificación de soluciones


In [ ]:
import import_ipynb
import CDP_0_SRC
from itertools import product
import community
import networkx as nx
import numpy as np

In [24]:
N = 1000
k_value = 4
G = CDP_0_SRC.crear_grafo()

def generate_random_partition_with_all_nodes(G, k):
    partition = {node: np.random.randint(1, k+1) for node in G.nodes()}
    return partition

sol_1 = generate_random_partition_with_all_nodes(G, k_value)
sol_2 = generate_random_partition_with_all_nodes(G, k_value)
sol_3 = generate_random_partition_with_all_nodes(G, k_value)

print("SOLUCIÓN 1")
print("Asignación de comunidades a nodos:")
print(sol_1)

print("\nSOLUCIÓN 2")
print("Asignación de comunidades a nodos:")
print(sol_2)

print("\nSOLUCIÓN 3")
print("Asignación de comunidades a nodos:")
print(sol_3)

SOLUCIÓN 1
Asignación de comunidades a nodos:
{'Nan Li': 4, 'Rong Jin': 4, 'Zhi-Hua Zhou': 3, 'Fajwel Fogel': 4, "Alexandre d'Aspremont": 1, 'Milan Vojnovic': 4, 'Nisheeth Srivastava': 4, 'Ed Vul': 4, 'Paul R. Schrater': 4, 'Sewoong Oh': 2, 'Devavrat Shah': 2, 'Siu On Chan': 2, 'Ilias Diakonikolas': 2, 'Rocco A. Servedio': 2, 'Xiaorui Sun': 3, 'Aaron van den Oord': 4, 'Benjamin Schrauwen': 3, 'Robert A. Vandermeulen': 2, 'Clayton Scott': 3, 'Qiang Liu': 3, 'Alexander T. Ihler': 3, 'Joseph G. Makin': 3, 'Philip N. Sabes': 2, 'Isabel Valera': 1, 'Zoubin Ghahramani': 2, 'Kumar Dubey': 3, 'Qirong Ho': 3, 'Sinead A. Williamson': 3, 'Eric P. Xing': 3, 'Anqi Wu': 4, 'Mijung Park': 2, 'Oluwasanmi O. Koyejo': 4, 'Jonathan W. Pillow': 2, 'Balaji Lakshminarayanan': 3, 'Daniel M. Roy': 4, 'Yee Whye Teh': 3, 'Jason Chang': 2, 'John W. Fisher III': 3, 'Mehmet G?nen': 1, 'Adam A. Margolin': 2, 'Qichao Que': 4, 'Mikhail Belkin': 2, 'Yusu Wang': 2, 'Bo Dai': 2, 'Bo Xie': 1, 'Niao He': 1, 'Yingyu Liang'

### Función objetivo


In [25]:
### Función objetivo
def modularidad(G, particion, weight='weight'):
    """
    Cálculo manual de la modularidad de una partición dada.
    """
    
    m = G.size(weight=weight)
    if m == 0:
        return 0
    
    grados = dict(G.degree(weight=weight))

    Q = 0.0
    for u, v, data in G.edges(data=True):
        w = data.get(weight, 1.0)
        if particion[u] == particion[v]:
            Q += w - (grados[u] * grados[v]) / (2 * m)
    
    fitness = Q / (2 * m)
    return fitness

## nuestra implementación
partition1 = dict(zip(G.nodes, sol_1))
partition2 = dict(zip(G.nodes, sol_2))
partition3 = dict(zip(G.nodes, sol_3))

print("Fitness solucion 1: {} ".format(modularidad(G, partition1)))
print("Fitness solucion 2: {} ".format(modularidad(G, partition2)))
print("Fitness solucion 3: {} ".format(modularidad(G, partition3)))

Fitness solucion 1: 0.0005574123709014806 
Fitness solucion 2: 0.0005574123709014806 
Fitness solucion 3: 0.0005574123709014806 
